### Importing the Libraries

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
tf.__version__ 

'2.13.0'

### Part 1 - Data Preprocessing

#### Generating images for the Training set

In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

#### Generating images for the Test set

In [4]:
test_datagen = ImageDataGenerator(rescale = 1./255)

### Creating the Training set

In [5]:
training_set = train_datagen.flow_from_directory("D:/dummyProject/newP/dataSet/train",                                
                                                 target_size = (128, 128),
                                                 batch_size = 10,
                                                 color_mode = 'grayscale',                                
                                                 class_mode = 'categorical')

Found 1017 images belonging to 3 classes.


In [6]:
test_set = test_datagen.flow_from_directory('D:/dummyProject/newP/dataSet/test',
                                            target_size = (128, 128),                                  
                                            batch_size = 10,        
                                            color_mode = 'grayscale',
                                            class_mode = 'categorical')

Found 453 images belonging to 3 classes.


### Part 2 - Building the CNN

#### Initializing the CNN

In [7]:
classifier = tf.keras.models.Sequential()

#### Step 1 - Convolution

In [8]:
classifier.add(tf.keras.layers.Conv2D(filters=32,
                                     kernel_size=3, 
                                     padding="same", 
                                     activation="relu", 
                                     input_shape=[128, 128, 1]))

#### Step 2 - Pooling

In [9]:
classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
                                         strides=2, 
                                         padding='valid'))

#### Adding a second convolutional layer

In [10]:
classifier.add(tf.keras.layers.Conv2D(filters=32, 
                                      kernel_size=3, 
                                      padding="same", 
                                      activation="relu"))

classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
                                         strides=2, 
                                         padding='valid'))

#### Step 3 - Flattening

In [11]:
classifier.add(tf.keras.layers.Flatten())

#### Step 4 - Full Connection

In [12]:
classifier.add(tf.keras.layers.Dense(units=128, 
                                     activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=96, activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=64, activation='relu'))
classifier.add(tf.keras.layers.Dense(units=3, activation='softmax')) # softmax for more than 2

### Part 3 - Training the CNN

#### Compiling the CNN

In [13]:
classifier.compile(optimizer = 'adam', 
                   loss = 'categorical_crossentropy', 
                   metrics = ['accuracy'])

#### Training the CNN on the Training set and evaluating it on the Test set

In [14]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 64, 64, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 32, 32, 32)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 32768)             0         
                                                                 
 dense (Dense)               (None, 128)               4

In [15]:
classifier.fit(training_set,
               epochs = 5,
               validation_data = test_set)

Epoch 1/5
102/102 [==============================] - 24s 210ms/step - loss: 0.8430 - accuracy: 0.5831 - val_loss: 0.1513 - val_accuracy: 1.0000
Epoch 2/5
102/102 [==============================] - 14s 137ms/step - loss: 0.1999 - accuracy: 0.9233 - val_loss: 0.0025 - val_accuracy: 1.0000
Epoch 3/5
102/102 [==============================] - 15s 144ms/step - loss: 0.0782 - accuracy: 0.9784 - val_loss: 1.5968e-04 - val_accuracy: 1.0000
Epoch 4/5
102/102 [==============================] - 14s 141ms/step - loss: 0.0522 - accuracy: 0.9823 - val_loss: 2.3082e-05 - val_accuracy: 1.0000
Epoch 5/5
102/102 [==============================] - 15s 143ms/step - loss: 0.0228 - accuracy: 0.9892 - val_loss: 8.8130e-07 - val_accuracy: 1.0000


#### Saving the Model

In [16]:
model_json = classifier.to_json()
with open("model-bw_smn.json", "w") as json_file:
    json_file.write(model_json)
print('Model Saved')
classifier.save_weights('model-bw_smn.h5')
print('Weights saved')

Model Saved
Weights saved
